# Model-based RL Practice

# -1. Setting

If you run in jupyter, turn 

```
colab = False
```

In [1]:
colab = True
if colab:
    !pip install gym pyvirtualdisplay > /dev/null 2>&1
    !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
    !apt-get update > /dev/null 2>&1
    !apt-get install cmake > /dev/null 2>&1
    !pip install --upgrade setuptools 2>&1
    !pip install ez_setup > /dev/null 2>&1

     |████████████████████████████████| 819 kB 9.8 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.2.0
    Uninstalling setuptools-57.2.0:
      Successfully uninstalled setuptools-57.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
if colab:
    from google.colab import drive
    drive.mount('/content/drive')

    %cd /content/drive/MyDrive/rl-master/rl-master/day5/mbrl
    !ls

Mounted at /content/drive
/content/drive/MyDrive/rl-master/rl-master/day5/mbrl
clockwise.png  memory.py  my_pendulum.py  reward_ftns.py
mb_agent.py    mpc.ipynb  __pycache__	  video


In [3]:
import numpy as np
from scipy.stats import truncnorm
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.nn import MSELoss
from my_pendulum import MyPendulumEnv
from reward_ftns import pendulum_reward
from memory import TransitionMemory

# 0. What we need for Model-based RL?

## 0.0. Of course, a model of true dynamics...
\begin{equation*}
    f_\theta(s, a) \approx f(s, a).
\end{equation*}
Then, given $s_t$ and $a_t$, our prediction of $s_{t+1}$ is given as follows:
\begin{equation*}
s_{t+1} = s_t + f_\theta(s_t, a_t).
\end{equation*}

In [4]:
class TransitionModel(nn.Module):
    def __init__(self, state_dim, act_dim, hidden1, hidden2):
        super(TransitionModel, self).__init__()
        self.state_dim = state_dim
        self.act_dim = act_dim
        self.fc1 = nn.Linear(state_dim + act_dim, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, state_dim)

    def forward(self, state, act):
        x = torch.cat([state, act], dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        delta = self.fc3(x)
        # TODO : fill the below line!
        next_state = state + delta
        
        return next_state

# 1. Key Components : Model Training & Model Predictive Control

In [5]:
class ModelBasedAgent:
    def __init__(self,
                 state_dim,
                 act_dim,
                 ctrl_range,
                 reward_ftn,
                 hidden1=400,
                 hidden2=400,
                 lr=0.001,
                 mem_sz=20000
                 ):
        self.dimS = state_dim
        self.dimA = act_dim
        self.ctrl_range = ctrl_range
        self.model = TransitionModel(state_dim, act_dim, hidden1, hidden2)
        self.memory = TransitionMemory(state_dim, act_dim, maxlen=mem_sz)
        
        self.reward_model: callable = reward_ftn
        
        self.optimizer = Adam(self.model.parameters(), lr=lr)
        
        self.alpha = 0.5    # smoothing parameter of cross-entropy optimization
        self.Ne = 20        # number of elite samples

    def train(self, batch_size):
        self.model.train()
        # note that training of the dynamics does not depend on any reward info
        (state_batch, act_batch, next_state_batch) = self.memory.sample_batch(batch_size)

        state_batch = torch.tensor(state_batch).float()
        act_batch = torch.tensor(act_batch).float()
        next_state_batch = torch.tensor(next_state_batch).float()

        prediction = self.model(state_batch, act_batch)


        loss_ftn = MSELoss()

        # TODO : What is the training loss for the neural network model?
        loss = loss_ftn(next_state_batch, prediction)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        loss_val = loss.detach().numpy()

        return loss_val

    def act(self, state, H, N, cross_entropy=False):
        if cross_entropy:
            return self.solve_cross_entropy_opt(state, H, N)
        else:
            return self.solve_random_shooting_opt(state, H, N)
    
    
    def solve_random_shooting_opt(self, state, H, N):
        """
        # generate N trajectories using the model of dynamics and random action sampling, and perform MPC
        # Remark! N roll-outs can be done simultaneously!

        given a state, execute an action based on random-sampling shooting method
        :param state: current state(numpy array)
        :param rew_ftn: vectorized reward function
        :param N: number of candidate action sequences to generate
        :param H: length of time horizon
        :return: action to be executed(numpy array)
        """
        assert N > 0 and H > 0

        dimA = self.dimA

        self.model.eval()

        states = np.tile(state, (N, 1)) # shape = (N, dim S)
        scores = np.zeros(N)    # array which contains cumulative rewards of roll-outs

        # generate K random action sequences of length H
        action_sequences = self.ctrl_range * (2. * np.random.rand(H, N, dimA) - 1.)
        # a ~ [-A, A]   a' ~ [0, 1]   (A * (2 * a' - 1)) ~ [-A, A]
        first_actions = action_sequences[0]     # shape = (N, dim A)

        for t in range(H):
            # s_t pred s_0  a_0 ->  prediction of s_1 -> a_1 -> pred s_2 ->
            actions = action_sequences[t]    # set of N actions, shape = (N, dim A)
            scores += self.reward_model(states, actions)

            s = torch.tensor(states).float()
            a = torch.tensor(actions).float()

            # TODO : Rollout action sequence to predict next state
            next_s = self.model(s, a)

            # torch tensor to numpy array
            # this cannot be skipped since a reward function takes numpy arrays as its inputs
            states = next_s.detach().numpy()

        best_seq = np.argmax(scores)
        # TODO : Complete our random-shooting MPC!
        best_action = first_actions[best_seq]
        return best_action
    
    
    
    def solve_cross_entropy_opt(self, state, H, N, rho=0.2, tol=1e-2, max_it=100):
        Ne = int(rho * N)
        dimA = self.dimA
        
        self.model.eval()

        states = np.tile(state, (N, 1)) # shape = (K, dim S)
        scores = np.zeros(N)    # array which contains cumulative rewards of roll-outs
        
        
        mu = np.zeros((H, 1, dimA))
        sigma = (.5 * self.ctrl_range) * np.ones((H, 1, dimA))
        
        assert np.max(sigma) >= tol
        best_action_so_far = None
        best_score_so_far = -np.inf
        trial = 0
        while np.max(sigma) >= tol and trial < max_it:
            action_sequences = truncnorm.rvs((-self.ctrl_range - mu) / sigma,
                                             (self.ctrl_range - mu) / sigma,
                                             loc=mu,
                                             scale=sigma,
                                             size=(H, N, dimA))
            # generate K random action sequences of length H
            for t in range(H):
                actions = action_sequences[t]    # set of K actions, shape = (K, dim A)
                scores += self.reward_model(states, actions)
                s = torch.tensor(states).float()
                a = torch.tensor(actions).float()
                next_s = self.model(s, a)
                # torch tensor to numpy array
                states = next_s.detach().numpy()
            # determine the elite samples of the group
            indices = np.argsort(scores)[-Ne:]
            idx = indices[-1]
            if scores[idx] > best_score_so_far:
                best_score_so_far = scores[idx]
                best_action_so_far = action_sequences[0, idx, :]   
            elite_samples = action_sequences[:, indices, :]
            mu += self.alpha * (np.mean(elite_samples, axis=1, keepdims=True) - mu)
            sigma += self.alpha * (np.std(elite_samples, axis=1, keepdims=True) - sigma)
            trial += 1
        if trial == max_it:
            warnings.warn('maximum iteration exceeded', RuntimeWarning)
        return best_action_so_far

In [6]:
def collect_rand_trajectories(env, transition_memory, num_trajectories):
    print('collecting random trajectories...')
    for i in range(num_trajectories):
        # collect random trajectories
        # able to be accelerated with mpi
        state = env.reset()
        for _ in range(200):
            action = env.action_space.sample()
            next_state, _, _, _ = env.step(action)
            transition_memory.append(state, action, next_state)
            state = next_state

        if i % 10 == 9:
            print('{} trajectories collected'.format(i + 1))
    print('done')
    return

# 2. Let's run the code!

In [7]:
env = MyPendulumEnv(g=10.0)
state_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]
ctrl_range = env.action_space.high[0]

In [8]:
agent = ModelBasedAgent(state_dim, act_dim, ctrl_range, pendulum_reward, hidden1=64, hidden2=64, mem_sz=30000)
collect_rand_trajectories(env, agent.memory, num_trajectories=150)

collecting random trajectories...
10 trajectories collected
20 trajectories collected
30 trajectories collected
40 trajectories collected
50 trajectories collected
60 trajectories collected
70 trajectories collected
80 trajectories collected
90 trajectories collected
100 trajectories collected
110 trajectories collected
120 trajectories collected
130 trajectories collected
140 trajectories collected
150 trajectories collected
done


In [9]:
batch_size = 256
max_iter = 10
num_ep_per_it = 10

In [10]:
# pre-train the network using randomly collected data
num_epochs = 20
epoch_size = len(agent.memory) // batch_size

for i in range(max_iter + num_epochs):
    # training loop
    # train the model
    if i == 0:
        print('pre-training...')

    if i == num_epochs:
        print('start MPC control...')

    if i < num_epochs:
        # first train the network only using randomly collected data
        for epoch in range(epoch_size):
            loss = agent.train(batch_size=batch_size)
        print('[iter {}] loss val : {:.4f}'.format(i, loss))

    else:
        scores = np.zeros(num_ep_per_it)
        
        for ep in range(num_ep_per_it):    
            state = env.reset()
            score = 0.
            for _ in range(200):
                if i % 5 == 0:
                    pass
                    # env.render()
                # environment roll-out
                # at each step, select an action using MPC (on-policy data)
                action = agent.act(state, H=60, N=600, cross_entropy=False)
                next_state, rew, _, _ = env.step(action)
                agent.memory.append(state, action, next_state)
                score += rew
                state = next_state
            scores[ep] = score
        avg = np.mean(scores)
        std = np.std(scores)
        env.close()
        print('score (over {} episodes) = {:4f}'.format(num_ep_per_it, avg), u'\u00B1', '{:4f}'.format(std))
        collect_rand_trajectories(env, agent.memory, num_trajectories=num_ep_per_it//2)
        for _ in range(num_ep_per_it):
            loss = agent.train(batch_size=batch_size)
        print('[iter {}] loss val : {:.4f}'.format(i, loss), end='\n')
        

pre-training...
[iter 0] loss val : 0.1009
[iter 1] loss val : 0.0357
[iter 2] loss val : 0.0269
[iter 3] loss val : 0.0266
[iter 4] loss val : 0.0231
[iter 5] loss val : 0.0182
[iter 6] loss val : 0.0364
[iter 7] loss val : 0.0212
[iter 8] loss val : 0.0212
[iter 9] loss val : 0.0144
[iter 10] loss val : 0.0128
[iter 11] loss val : 0.0174
[iter 12] loss val : 0.0190
[iter 13] loss val : 0.0118
[iter 14] loss val : 0.0159
[iter 15] loss val : 0.0104
[iter 16] loss val : 0.0209
[iter 17] loss val : 0.0146
[iter 18] loss val : 0.0160
[iter 19] loss val : 0.0132
start MPC control...
score (over 10 episodes) = -803.554498 ± 219.956601
collecting random trajectories...
done
[iter 20] loss val : 0.0205
score (over 10 episodes) = -840.072961 ± 391.135680
collecting random trajectories...
done
[iter 21] loss val : 0.0232
score (over 10 episodes) = -840.158664 ± 199.685257
collecting random trajectories...
done
[iter 22] loss val : 0.0223
score (over 10 episodes) = -865.739957 ± 358.946136
coll

In [11]:
if colab:
    import gym
    from gym.wrappers import Monitor
    import glob
    import io
    import base64
    from IPython.display import HTML
    from pyvirtualdisplay import Display
    from IPython import display as ipythondisplay

    display = Display(visible=0, size=(1400, 900))
    display.start()

    def show_video():
      mp4list = glob.glob('video/*.mp4')
      if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                </video>'''.format(encoded.decode('ascii'))))
      else: 
        print("Could not find video")
        

    def wrap_env(env):
      env = Monitor(env, './video', force=True)
      return env

    env = wrap_env(env)

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [13]:
env = MyPendulumEnv(g=10.0)
if colab:
  env = wrap_env(env)
obs = env.reset()

done = False
score = 0.

for _ in range(400):
    env.render()
    obs, rew, done, _ = env.step(agent.act(obs, H=60, N=400, cross_entropy=False))
    score += rew
    
env.close()
print('score : ', score)

if colab:
  show_video()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


score :  -2311.674409384208
